# Simulation

This simulation shows how the app should receive the accident information, and what will be the result

In [1]:
# Custom Libraries
import project.source.locator as loc
import project.source.mapquest as mq 

## Receiving an accident alert

In [2]:
# This code block instances locator class to create Lat-Lon point and its buffer.
# The buffer will be used to determine what agents are capable to attend the call.

# The operator should type the address
address = 'Parque el Virrey, Bogotá'

# This radius should be calculated based on history
radius = 2000

locate = loc.Locator()
location = locate.get_location(address)
if location:
    accident_point = locate.make_point(address)
    buffer = locate.make_buffer(accident_point, radius)

print(f'''The Accident point is: {accident_point['geometry'][0]}\n\nThe polygon around the accident point is: {buffer[0]}''')

The Accident point is: POINT (-74.0520793152638 4.672132599999999)

The polygon around the accident point is: POLYGON ((-74.06563249536732 4.671148689129443, -74.06566200592043 4.672487112220043, -74.06556065042778 4.673821997361786, -74.06532940249299 4.675140496606276, -74.06497048521182 4.676429919244721, -74.06448734991469 4.677677853751212, -74.06388464312909 4.678872287069715, -74.06316816207463 4.680001720105179, -74.0623447991087 4.681055278313979, -74.06142247564655 4.682022816335533, -74.06041006618233 4.682895015659647, -74.05931731312953 4.683663474391597, -74.05815473329032 4.6843207882492, -74.05693351684201 4.684860622008503, -74.05566541980401 4.685277770705151, -74.05436265101203 4.685568209994171, -74.05303775467968 4.685729135174751, -74.05170348967378 4.685758988493517, -74.05037270666314 4.685657474452405, -74.04905822432204 4.685425562961615, -74.04777270578347 4.685065480295778, -74.04652853653427 4.684580687928855, -74.04533770493416 4.683975849441126, -74.04421

In [3]:
# Viewing the above information
import folium 

m = folium.Map((location.latitude, location.longitude), max_zoom=20, zoom_start=14)
folium.GeoJson(buffer).add_to(m)
folium.GeoJson(accident_point, tooltip=folium.GeoJsonTooltip(['address'])).add_to(m)
m

## Checking Agents

In [13]:
# Creating 10 Agents
agents = []

locate = loc.Locator()

for i in range(10):
    agents.append(locate.make_random_agent())


In [33]:
for agent in agents:
    if not agent['address'].isnull().any():
        print(agent)
    

                    geometry  \
0  POINT (-74.02584 4.67239)   

                                             address  
0  San Luis, UPZ San Luis, San Luis, Bogotá, Bogo...  
                    geometry  \
0  POINT (-74.19288 4.82674)   

                                             address  
0  Vía la Punta Tenjo, Madrid, Tenjo, Sabana Occi...  
                    geometry                                    address
0  POINT (-74.08595 4.40791)  Chipaque, Oriente, Cundinamarca, Colombia
                    geometry  \
0  POINT (-74.07178 4.63921)   

                                             address  
0  50-95, Carrera 20, Galerías, UPZ Galerías, Loc...  
                    geometry                                       address
0  POINT (-74.15320 4.82745)  Tenjo, Sabana Centro, Cundinamarca, Colombia
                    geometry                                       address
0  POINT (-74.15320 4.82745)  Tenjo, Sabana Centro, Cundinamarca, Colombia
                    geometry  \

In [34]:
m = folium.Map((location.latitude, location.longitude), max_zoom=20, zoom_start=14)
folium.GeoJson(buffer).add_to(m)
folium.GeoJson(accident_point, tooltip='Accident').add_to(m)
for agent in agents:
    if not agent['address'].isnull().any():
        folium.GeoJson(agent, tooltip='Agent').add_to(m)
m